# Finetuning a Pytorch Lightning Image Classifier

This example introduces how to train a Pytorch Lightning Module using AIR {class}`LightningTrainer <ray.train.lightning.LightningTrainer>`. We will demonstrate how to train a basic neural network on the MNIST dataset with distributed data parallelism.


In [2]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split, Subset
from torchmetrics import Accuracy
from torchvision.datasets import MNIST
from torchvision import transforms

import pytorch_lightning as pl
from pytorch_lightning import trainer
from pytorch_lightning.core import datamodule
from pytorch_lightning.loggers.csv_logs import CSVLogger

## Prepare Dataset and Module

The Pytorch Lightning Trainer takes either `torch.utils.data.DataLoader` or `pl.LightningDataModule` as data inputs. You can keep using them without any changes for the Ray AIR LightningTrainer. 

In [3]:
class MNISTDataModule(pl.LightningDataModule):
    def __init__(self, batch_size=100):
        super().__init__()
        self.data_dir = os.getcwd()
        self.batch_size = batch_size
        self.transform = transforms.Compose(
            [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
        )

    def setup(self, stage=None):
        # split data into train and val sets
        mnist = MNIST(
            self.data_dir, train=True, download=True, transform=self.transform
        )
        self.mnist_train, self.mnist_val = random_split(mnist, [55000, 5000])

    def train_dataloader(self):
        return DataLoader(self.mnist_train, batch_size=self.batch_size, num_workers=4)

    def val_dataloader(self):
        return DataLoader(self.mnist_val, batch_size=self.batch_size, num_workers=4)

    def test_dataloader(self):
        self.mnist_test = MNIST(
            self.data_dir, train=False, download=True, transform=self.transform
        )
        return DataLoader(self.mnist_test, batch_size=self.batch_size, num_workers=4)

datamodule = MNISTDataModule(batch_size=256)

Next, define a simple multi-layer perception as the subclass of `pl.LightningModule`.

In [4]:
class MNISTClassifier(pl.LightningModule):
    def __init__(self, lr=1e-3):
        super(MNISTClassifier, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 128),
            nn.ReLU(),
            nn.Linear(128, 10),
            nn.ReLU(),
        )
        self.lr = lr
        self.accuracy = Accuracy()

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = self.linear_relu_stack(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = torch.nn.functional.cross_entropy(y_hat, y)
        self.log("train_loss", loss)
        return loss

    def validation_step(self, val_batch, batch_idx):
        loss, acc = self._shared_eval(val_batch)
        self.log("val_accuracy", acc)
        return {"val_loss": loss, "val_accuracy": acc}
    
    def test_step(self, test_batch, batch_idx):
        loss, acc = self._shared_eval(test_batch)
        self.log("test_accuracy", acc)
        return {"test_loss": loss, "test_accuracy": acc}
    
    def _shared_eval(self, batch):
        x, y = batch
        logits = self.forward(x)
        loss = F.nll_loss(logits, y)
        acc = self.accuracy(logits, y)
        return loss, acc

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        avg_acc = torch.stack([x["val_accuracy"] for x in outputs]).mean()
        self.log("val_loss", avg_loss, sync_dist=True)
        self.log("val_accuracy", avg_acc, sync_dist=True)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        return optimizer

You don't need to make any change to the definition of PyTorch Lightning model and datamodule.

## Define the Cofigurations for AIR LightningTrainer

The {meth}`LightningConfigBuilder <ray.train.lightning.LightningConfigBuilder>` class stores all the parameters involved in training a PyTorch Lightning module. It takes the same parameter lists as those in PyTorch Lightning.

- The `.module()` method takes a subclass of `pl.LightningModule` and its initialization parameters. `LightningTrainer` will instantiate a model instance internally in the workers' training loop.
- The `.trainer()` method takes the initialization parameters of `pl.Trainer`. You can specify training configurations, loggers, and callbacks here.
- The `.fit_params()` method stores all the parameters that will be passed into `pl.Trainer.fit()`, including train/val dataloaders, datamodules, and checkpoint paths.
- The `.checkpointing()` method saves the configurations for a `ModelCheckpoint` callback. Note that the `LightningTrainer` reports the latest metrics to the AIR session when a new checkpoint is saved.
- The .build() method generates a dictionary that contains all the configurations in the builder. This dictionary will be passed to `LightningTrainer` later.

In [5]:
from ray.air.config import RunConfig, ScalingConfig, CheckpointConfig
from ray.train.lightning import LightningTrainer, LightningConfigBuilder

lightning_config = (
    LightningConfigBuilder()
    .module(
        MNISTClassifier, lr=1e-3
    )
    .trainer(max_epochs=10, accelerator="cpu", log_every_n_steps=100, logger=CSVLogger("logs"))
    .fit_params(datamodule=datamodule)
    .checkpointing(monitor="val_accuracy", mode="max", save_top_k=3)
    .build()
)

In [6]:
scaling_config = ScalingConfig(
    num_workers=4, use_gpu=False, resources_per_worker={"CPU": 1}
)

run_config = RunConfig(
    name="ptl-mnist-example",
    local_dir="/tmp/ray_results",
    checkpoint_config=CheckpointConfig(num_to_keep=3, checkpoint_score_attribute="val_accuracy", checkpoint_score_order="max")
)

In [7]:
trainer = LightningTrainer(
    lightning_config=lightning_config,
    scaling_config=scaling_config,
    run_config=run_config,
)

Now fit your trainer:

In [8]:
result = trainer.fit()
print("Validation Accuracy: ", result.metrics["val_accuracy"])
result

find: ‘.git’: No such file or directory
2023-03-21 18:10:34,748	INFO worker.py:1362 -- Connecting to existing Ray cluster at address: 10.0.26.151:6379...
2023-03-21 18:10:34,757	INFO worker.py:1550 -- Connected to Ray cluster. View the dashboard at https://console.anyscale-staging.com/api/v2/sessions/ses_lakhijrn6mdv9hqpld5trwzlxz/services?redirect_to=dashboard 
2023-03-21 18:10:35,521	INFO packaging.py:346 -- Pushing file package 'gcs://_ray_pkg_118796b8d144b5c957c7ad556eb319a1.zip' (52.44MiB) to Ray cluster...
2023-03-21 18:10:36,431	INFO packaging.py:359 -- Successfully pushed file package 'gcs://_ray_pkg_118796b8d144b5c957c7ad556eb319a1.zip'.
/home/ray/anaconda3/lib/python3.8/site-packages/ray/tune/experiment/experiment.py:170: UserWarning: The `local_dir` argument of `Experiment is deprecated. Use `storage_path` or set the `TUNE_RESULT_DIR` environment variable instead.
  warnings.warn(


(RayTrainWorker pid=259202) 2023-03-21 18:10:48,890	INFO config.py:86 -- Setting up process group for: env:// [rank=0, world_size=4]
(RayTrainWorker pid=259205) Missing logger folder: logs/lightning_logs
(RayTrainWorker pid=259204) Missing logger folder: logs/lightning_logs
(RayTrainWorker pid=259202) GPU available: False, used: False
(RayTrainWorker pid=259202) TPU available: False, using: 0 TPU cores
(RayTrainWorker pid=259202) IPU available: False, using: 0 IPUs
(RayTrainWorker pid=259202) HPU available: False, using: 0 HPUs
(RayTrainWorker pid=259202) Missing logger folder: logs/lightning_logs
(RayTrainWorker pid=259203) Missing logger folder: logs/lightning_logs
(RayTrainWorker pid=259202) 
(RayTrainWorker pid=259202)   | Name              | Type       | Params
(RayTrainWorker pid=259202) -------------------------------------------------
(RayTrainWorker pid=259202) 0 | linear_relu_stack | Sequential | 101 K 
(RayTrainWorker pid=259202) 1 | accuracy          | Accuracy   | 0     
(

Trial name,_report_on,date,done,epoch,experiment_tag,hostname,iterations_since_restore,node_ip,pid,should_checkpoint,step,time_since_restore,time_this_iter_s,time_total_s,timestamp,train_loss,training_iteration,trial_id,val_accuracy,val_loss
LightningTrainer_5a3f6_00000,train_epoch_end,2023-03-21_18-11-13,True,9,0,ip-10-0-26-151,10,10.0.26.151,258943,True,540,28.1988,2.00198,28.1988,1679447473,0.038871,10,5a3f6_00000,0.976058,-11.287


(RayTrainWorker pid=259202) `Trainer.fit` stopped: `max_epochs=10` reached.
2023-03-21 18:11:15,922	INFO tune.py:817 -- Total run time: 39.40 seconds (39.32 seconds for the tuning loop).


Validation Accuracy:  0.9760578274726868


Result(
  metrics={'_report_on': 'train_epoch_end', 'train_loss': 0.03887101635336876, 'val_accuracy': 0.9760578274726868, 'val_loss': -11.286975860595703, 'epoch': 9, 'step': 540, 'should_checkpoint': True, 'done': True, 'trial_id': '5a3f6_00000', 'experiment_tag': '0'},
  log_dir=PosixPath('/tmp/ray_results/ptl-mnist-example/LightningTrainer_5a3f6_00000_0_2023-03-21_18-10-38'),
  checkpoint=LightningCheckpoint(local_path=/tmp/ray_results/ptl-mnist-example/LightningTrainer_5a3f6_00000_0_2023-03-21_18-10-38/checkpoint_000009)
)

## Test your network on the test data

Next, we use PyTorch Lightning's native interface to evaluate the best model: To run the test loop using the test_step() method, simply pass the trained model to trainer.test(). 

For faster inference on large datasets, you can try to use AIR's {class}`BatchPredictor <ray.train.batch_prediction.BatchPredictor>`.

In [9]:
checkpoint = result.best_checkpoints[0][0]
best_model = checkpoint.get_model(MNISTClassifier)

In [10]:
trainer = pl.Trainer()
result = trainer.test(best_model, dataloaders=datamodule.test_dataloader())

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
2023-03-21 18:11:16.060942: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-21 18:11:17.107466: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-21 18:11:17.107538: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: canno

Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.9656000137329102     │
└───────────────────────────┴───────────────────────────┘